In [1]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using TensorFlow backend.


In [2]:
path = get_file('nietzsche.txt', origin = "https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read().lower();
print('corpus length:', len(text))

corpus length: 600901


In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars) + 1
print('total chars:', vocab_size)

total chars: 60


In [4]:
chars.insert(0, "\o")

In [5]:
''.join(chars[1:-6])

'\n !"\'(),-.0123456789:;=?[]_abcdefghijklmnopqrstuvwxyz'

In [6]:
char_indices = dict((c, i) for i, c in enumerate(chars)) #index for every char
indices_char = dict((i, c) for i, c in enumerate(chars)) #indices to char mapping

In [7]:
idx = [char_indices[c] for c in text]

In [8]:
idx[:10]

[43, 45, 32, 33, 28, 30, 32, 1, 1, 1]

In [9]:
''.join(indices_char[i] for i in idx[:70])

'preface\n\n\nsupposing that truth is a woman--what then? is there not gro'

In [10]:
#create a list of every 4th characters at the 0th, 1st, 2nd, 3rd charcters
cs = 3
c1_dat = [idx[i]   for i in range(0, len(idx)-1-cs, cs)]
c2_dat = [idx[i+1] for i in range(0, len(idx)-1-cs, cs)]
c3_dat = [idx[i+2] for i in range(0, len(idx)-1-cs, cs)]
c4_dat = [idx[i+3] for i in range(0, len(idx)-1-cs, cs)]

In [11]:
#inputs
x1 = np.stack(c1_dat[:-2])
x2 = np.stack(c2_dat[:-2])
x3 = np.stack(c3_dat[:-2])

In [12]:
#outputs
y = np.stack(c4_dat[:-2])

In [13]:
x1[:4], x2[:4], x3[:4]

(array([43, 33, 32,  1]), array([45, 28,  1, 46]), array([32, 30,  1, 48]))

In [14]:
y[:4]

array([33, 32,  1, 43])

In [15]:
x1.shape, y.shape

((200297,), (200297,))

In [16]:
n_fac = 42

In [17]:
def embedding_input(name, n_in, n_out):
    inp = Input(shape=(1,), dtype='int64', name=name)
    emb = Embedding(n_in, n_out, input_length=1)(inp)
    return inp, Flatten()(emb)

In [18]:
c1_in, c1 = embedding_input("c1", vocab_size, n_fac)
c2_in, c2 = embedding_input("c2", vocab_size, n_fac)
c3_in, c3 = embedding_input("c3", vocab_size, n_fac)

In [19]:
n_hidden = 256

In [20]:
dense_in = Dense(n_hidden, activation='relu')
c1_hidden = dense_in(c1)
dense_hidden = Dense(n_hidden, activation='tanh')

In [21]:
c2_dense = dense_in(c2)
hidden_c1 = dense_hidden(c1_hidden)
c12_hidden = merge([c2_dense, hidden_c1])

/Users/trinakarmakar/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  app.launch_new_instance()
/Users/trinakarmakar/anaconda2/lib/python2.7/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [22]:
c3_dense = dense_in(c3)
hidden_c12 = dense_hidden(c12_hidden)
c123_hidden = merge([c3_dense, hidden_c12])

/Users/trinakarmakar/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  app.launch_new_instance()


In [23]:
dense_out = Dense(vocab_size, activation='softmax')
c4_out = dense_out(c123_hidden)

In [24]:
model = Model([c1_in, c2_in, c3_in], c4_out)

In [25]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer=Adam(lr=0.01))

In [26]:
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=4)

/Users/trinakarmakar/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if __name__ == '__main__':


Epoch 1/4
200297/200297 [==============================] - 27s - loss: 2.4441       

## Test model

In [27]:
def get_next(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = [np.array(i)[np.newaxis] for i in idxs]
    p = model.predict(arrs)
    i = np.argmax(p)
    return chars[i]

In [28]:
get_next('phi')

'm'

In [29]:
get_next(' th')

'e'

In [30]:
get_next(' an')

'd'

In [31]:
cs = 8

In [32]:
#For each 0 through 7 create a list of 8th character with the starting point. These 
#will be 8 input to our model

c_in_dat = [[idx[i+n] for i in range(0, len(idx)-1-cs, cs)] 
            for n in range(cs)]

In [33]:
#create a list of next character for each of those charcter which will be label 
# for our input data
c_out_dat = [idx[i+cs] for i in range(0, len(idx)-1-cs, cs)]

In [34]:
xs = [np.stack(c[:-2]) for c in c_in_dat]

In [35]:
len(xs), xs[0].shape

(8, (75110,))

In [36]:
y = np.stack(c_out_dat[:-2])

So each column bellow is one series of 8 characters from the text

In [37]:
[xs[n][:cs] for n in range(cs)]

[array([43,  1, 36,  2, 46, 41, 47,  2]),
 array([45,  1, 41, 47,  2,  9, 35, 47]),
 array([32, 46, 34, 45, 28,  9, 32, 35]),
 array([33, 48,  2, 48,  2, 50, 41, 32]),
 array([28, 43, 47, 47, 50, 35, 24, 45]),
 array([30, 43, 35, 35, 42, 28,  2, 32]),
 array([32, 42, 28,  2, 40, 47, 36,  2]),
 array([ 1, 46, 47, 36, 28,  2, 46, 41])]

In [38]:
y[:cs]

array([ 1, 36,  2, 46, 41, 47,  2, 42])

In [39]:
n_fac = 42

In [40]:
def embedding(name, n_in, n_out):
    inp = Input(shape=(1,), dtype='int64', name=name+'_in')
    emb = Embedding(n_in, n_out, input_length=1, name = name+'_emb')(inp)
    return inp, Flatten()(emb)

In [41]:
c_ins = [embedding_input('c'+str(n), vocab_size, n_fac) for n in range(cs)]

In [42]:
n_hidden=256

In [43]:
dense_in = Dense(n_hidden, activation='relu')
dense_hidden=Dense(n_hidden, activation='relu', init = 'identity')
dense_out = Dense(vocab_size, activation='softmax')

/Users/trinakarmakar/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(256, kernel_initializer="identity", activation="relu")`
  from ipykernel import kernelapp as app


In [44]:
hidden = dense_in(c_ins[0][1])

In [45]:
for i in range(1, cs):
    c_dense = dense_in(c_ins[i][1])
    hidden = dense_hidden(hidden)
    hidden = merge([c_dense, hidden])

/Users/trinakarmakar/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [46]:
c_out = dense_out(hidden)

In [47]:
model = Model([c[0] for c in c_ins], c_out)

In [48]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer=Adam(lr=0.01))

In [49]:
model.fit(xs, y, batch_size=64, nb_epoch=8)

/Users/trinakarmakar/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if __name__ == '__main__':


Epoch 1/8
75110/75110 [==============================] - 21s - loss: 2.4126       

In [50]:
def get_next(inp):
    idxs = [np.array(char_indices[c])[np.newaxis] for c in inp]
    p = model.predict(idxs)
    return chars[np.argmax(p)]

In [51]:
get_next('for thos')

' '

In [52]:
get_next('part of ')

't'

In [53]:
get_next('queens a')

'n'

## Our first RNN with keras

In [54]:
n_hidden, n_fac, cs , vocab_size = (256, 42, 8, 86)

In [55]:
model = Sequential([
    Embedding(vocab_size, n_fac, input_length=cs),
    SimpleRNN(n_hidden, activation='relu', inner_init='identity'),
    Dense(vocab_size, activation='softmax')
])

/Users/trinakarmakar/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `SimpleRNN` call to the Keras 2 API: `SimpleRNN(256, activation="relu", recurrent_initializer="identity")`
  app.launch_new_instance()


In [56]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 8, 42)             3612      
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 256)               76544     
_________________________________________________________________
dense_7 (Dense)              (None, 86)                22102     
Total params: 102,258
Trainable params: 102,258
Non-trainable params: 0
_________________________________________________________________


In [57]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.001))

In [ ]:
model.fit(np.concatenate(xs, axis=1), y, batch_size=64, nb_epoch=8)

/Users/trinakarmakar/anaconda2/lib/python2.7/site-packages/keras/models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/8
75110/75110 [==============================] - 29s - loss: 2.7031      

In [59]:
def get_next_keras(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = np.array[idxs][np.newacis,:]
    p = model.predict(arrs)[0]
    return chars[np.argmax(p)]

In [60]:
get_next_keras('this is ')
get_next_keras('part of ')
get_next_keras('queens a')

TypeError: 'builtin_function_or_method' object has no attribute '__getitem__'

## Returning Sequences:
    

In this case we can keep input unchaged but chage output as input but shifted by ne character

In [ ]:
c_out_dat = [[idx[i+n] for i in range(1, len(idx)-cs, cs)] for n in range(cs)]

In [ ]:
ys = [np.stack(c[:-2]) for c in c_out_dat]

In [ ]:
[xs[n][:cs] for n in range(cs)]

In [ ]:
[ys[n][:cs] for n in range(cs)]

In [ ]:
dense_in = Dense(nhidden, activation='relu')
dense_hidden = Dense(n_hidden, activation='relu', init='identity')
dense_out = Dense(vocab_size, activation='softmax', name='output')

In [ ]:
inp1 = Input(shape=(n_fac,), name='zeros'
hidden = dense_in(inp1)

In [ ]:
outs = []

for i in range(cs):
    c_dense = dense_in(c_ins[i][1])
    hidden = dense_hidden(hidden)
    hidden = merge([c_dense, hidden], model='sum')
    #every layer now has an output
    outs.append(dense_out(hidden))

In [ ]:
model = Model([inp1] + [c[0] for c in c_ins], outs)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.001))

In [ ]:
zeros = np.tile(np.zeros(n_fac), (len(xs[0]), 1))
zeros.shape

In [ ]:
model.fit([zeros]+xs, ys, batch_size=64, nb_epoch=8)

In [ ]:
def get_netxs(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = [np.array(i)[np.newaxis] for i in idxs]
    p = model.predict([np.zeros(n_fac)[np.newaxis, :]] + arrs)
    print(list(inp))
    return [chars[np.argmax(o)] for o in p]

In [ ]:
get_nexts(' this is')
get_nexts(' part of')

## Sequence models with Keras

In [ ]:
n_hidden, n_fac, cs, vocab_size

In [ ]:
model = Sequential([Embedding(vocab_size, n_fac, input_length=cs),
                   SimpleRNN(n_hidden, return_sequences, 
                             activation='relu', inner_init='identity'),
                    
                    TimeDistributed(Dense(vocab_size, activation='softmax'))
                   ])

In [ ]:
model.summary

In [ ]:
model.compile(loss = 'sparse_categotical_crossentropy', optimizer=Adam(lr=0.001))

In [ ]:
xs[0].shape

In [ ]:
x_rnn = np.stack(np.squeeze(xs), axis=1)
y_rnn=np.atleast_3d(np.stack(ys, axis=1))

In [ ]:
x_rnn.shape, y_rnn.shape

In [ ]:
model.fit(x_rnn, y_rnn, batch_size=64, nb_epoch=8)

In [ ]:
def get_next_keras(inp):
    idxs = [char_indices[c] for c in inp]
    arr = np.array(idxs)[np.newaxis,:]
    p = model.predict(arr)[0]
    print(list(inp))
    return [chars[np.argmax[o]] for o in p]

In [ ]:
get_next_keras( 'this is')